# Mongo DB Scripts

Various explorations of MongoDB python client to create business logic for Stay Presnet application

## Pandas Groupby of application

In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [22]:
d = pd.read_csv('hi.csv')

In [23]:
d.fillna(0, inplace = True)

In [30]:
d.groupby('application')['numClients'].sum().reset_index()

,application,numClients
0,Antivirus updates,10
1,Blogger,9
2,CDNs,117
3,Dropbox,9
4,ESPN,1
5,Encrypted TCP (SSL),40
6,Facebook,372
7,Gmail,195
8,Google,216
9,Google Drive,192


## Creation of Student Identities mapped to MAC Addresses

`loc` was a dataframe of a sample of location data we recieved from a Meraki device.

In [ ]:
macIDs = loc['clientMac'].unique()

In [80]:
# Connection to client
db = MongoClient('mongodb://<username>:<password>@ds044689.mlab.com:44689/meraki')

In [81]:
# Students Mongodb collection
students = db.meraki.students

In [83]:
# Going through indices and setting student identities.
for i in range(len(macIDs)):
    students.insert_one({"SID": i, 'name':'Student' + str(i), 'clientMac': macIDs[i]})

In [139]:
mac_address = 'f4:0f:24:36:15:3d'

In [111]:
students.find({'clientMac': mac_address})[0]

{'SID': 9,
 '_id': ObjectId('59d9f70d5e948f435621f800'),
 'clientMac': 'a0:99:9b:0e:95:4c',
 'name': 'Student9'}

In [90]:
checkins = db.meraki.locations

## Data Parsing

Some date parsing to work on student attendance logic.

In [93]:
from datetime import datetime
parse_date = lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%SZ')

In [140]:
times = [parse_date(x['seenTime']) for x in list(checkins.find({'clientMac': mac_address}))]

In [97]:
time_str[0]

'2017-10-08T06:02:01Z'

In [141]:
times

[datetime.datetime(2017, 10, 8, 8, 53, 56),
 datetime.datetime(2017, 10, 8, 8, 54, 33),
 datetime.datetime(2017, 10, 8, 8, 54, 39),
 datetime.datetime(2017, 10, 8, 8, 54, 58),
 datetime.datetime(2017, 10, 8, 8, 55, 20),
 datetime.datetime(2017, 10, 8, 8, 55, 27),
 datetime.datetime(2017, 10, 8, 8, 55, 53),
 datetime.datetime(2017, 10, 8, 8, 55, 53),
 datetime.datetime(2017, 10, 8, 8, 55, 52),
 datetime.datetime(2017, 10, 8, 8, 55, 42),
 datetime.datetime(2017, 10, 8, 8, 55, 51),
 datetime.datetime(2017, 10, 8, 9, 9, 53),
 datetime.datetime(2017, 10, 8, 9, 9, 54),
 datetime.datetime(2017, 10, 8, 9, 9, 58),
 datetime.datetime(2017, 10, 8, 9, 9, 58),
 datetime.datetime(2017, 10, 8, 9, 10, 11),
 datetime.datetime(2017, 10, 8, 9, 10, 6),
 datetime.datetime(2017, 10, 8, 9, 10, 11),
 datetime.datetime(2017, 10, 8, 9, 10, 12),
 datetime.datetime(2017, 10, 8, 9, 12, 8),
 datetime.datetime(2017, 10, 8, 9, 12, 46),
 datetime.datetime(2017, 10, 8, 9, 12, 59),
 datetime.datetime(2017, 10, 8, 9, 13,

In [142]:
max(times)

datetime.datetime(2017, 10, 8, 10, 37, 31)

In [143]:
min(times)

datetime.datetime(2017, 10, 8, 8, 53, 56)

In [144]:
# Time difference of a certain mac address on meraki network
t_d = max(times) - min(times)

In [151]:
t_d.seconds

6215